In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Parameters 
learning_rate = 0.00001
epochs = 10
batch_size = 128

In [4]:
# Number of samples to calculate validation and accuracy
test_valid_size = 256

In [5]:
# Network Parameters
n_classes = 10 
dropout = 0.75

In [6]:
# Store layers weights & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024,n_classes]))
}

In [7]:
biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

In [10]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'],biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1,weights['out']), biases['out'])
    return out

In [11]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy 
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x:batch_x,
                y:batch_y,
                keep_prob:dropout
            })
            
            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x:batch_x,
                y:batch_y,
                keep_prob:1.
            })
            valid_acc = sess.run(accuracy, feed_dict={
                x:mnist.validation.images[:test_valid_size],
                y:mnist.validation.labels[:test_valid_size],
                keep_prob:1
            })
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch+1,batch+1,loss,valid_acc))
            
            # Calculate Test Accuracy
            test_acc = sess.run(accuracy, feed_dict={
                x: mnist.test.images[:test_valid_size],
                y: mnist.test.labels[:test_valid_size],
                keep_prob:1.
            })
            print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 46797.3086 Validation Accuracy: 0.085938
Testing Accuracy: 0.078125
Epoch  1, Batch   2 -Loss: 38964.5234 Validation Accuracy: 0.082031
Testing Accuracy: 0.08203125
Epoch  1, Batch   3 -Loss: 33280.5586 Validation Accuracy: 0.070312
Testing Accuracy: 0.0859375
Epoch  1, Batch   4 -Loss: 33504.0781 Validation Accuracy: 0.089844
Testing Accuracy: 0.08984375
Epoch  1, Batch   5 -Loss: 29010.0898 Validation Accuracy: 0.093750
Testing Accuracy: 0.109375
Epoch  1, Batch   6 -Loss: 25845.3984 Validation Accuracy: 0.105469
Testing Accuracy: 0.1171875
Epoch  1, Batch   7 -Loss: 26895.5527 Validation Accuracy: 0.132812
Testing Accuracy: 0.14453125
Epoch  1, Batch   8 -Loss: 23644.5469 Validation Accuracy: 0.136719
Testing Accuracy: 0.140625
Epoch  1, Batch   9 -Loss: 24034.1680 Validation Accuracy: 0.128906
Testing Accuracy: 0.15234375
Epoch  1, Batch  10 -Loss: 22470.8164 Validation Accuracy: 0.140625
Testing Accuracy: 0.1484375
Epoch  1, Batch  11 -Loss: 20934.1094 V